In [ ]:
from IPython.lib.display import YouTubeVideo
from sklearn.linear_model import RidgeCV, Ridge, LassoCV, LinearRegression,Lasso
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math

df = pd.read_excel('pH Relation.xlsx')
#Clean DATE
df['DATE'] = pd.to_datetime(df['DATE'])
df['DATE'] = df['DATE'].dt.date

#Delete blank
df = df.dropna(subset=['pH'], axis=0)
df = df.dropna(subset=['Ammonia/ppm'], axis=0)
df = df.dropna(subset=['Sulfide/ppm'], axis=0)
df = df.dropna(subset=['Chloride/ppm'], axis=0)
# #Clean crude diet
# label_encoder = LabelEncoder()
# df['Numerical_diet'] = label_encoder.fit_transform(df['Crude diet'])

# df['H+'] = df['pH'].apply(lambda x: math.exp(-x))
# df['Ammonia/ppb'] = df['Ammonia/ppm']*1000
# df['Sulfide/ppb'] = df['Sulfide/ppm']*1000
# df['Chloride/ppb'] = df['Chloride/ppm']*1000
#resetindex
df.reset_index(drop=True, inplace=True)
#plot
# sns.scatterplot(data = df, x = "pH", y='Ammonia/ppm')
# plt.show()
# sns.scatterplot(data = df, x = "pH", y='Sulfide/ppm')
# plt.show()
# sns.scatterplot(data = df, x = "pH", y='Chloride/ppm')
# plt.show()




In [ ]:
print(df)

           DATE  IOW(In/Out)    pH  Ammonia/ppm  Sulfide/ppm  Chloride/ppm   
0    2022-12-06            0   5.5          2.0        0.034        1.0200  \
1    2022-12-07            0   5.5          9.0        0.088        1.0280   
2    2022-12-22            1  8.58         18.0        0.070        0.7090   
3    2022-12-23            1  8.58         18.0        0.070        0.7090   
4    2022-12-25            1  8.35         56.0        0.120        1.7564   
..          ...          ...   ...          ...          ...           ...   
134  2023-08-27            0  5.87         71.0        2.000        6.2000   
135  2023-08-28            0  6.67         30.0        0.020        6.9127   
136  2023-08-30            0  7.98         52.0        0.060        6.2037   
137  2023-09-01            1  8.51         74.0        0.040        4.4312   
138  2023-09-04            1     9         90.0        0.020        6.9127   

     Iron/ppm  Oil&Grease/ppm  DO/ppm   
0       0.500         

In [ ]:
df2 = df.iloc[:,8:11]
df2.head()

,DO/ppm,Crude diet,Crude ratio
0,6.0,BEN-WTM-UJU-PAT-SIB-SHGO-GIR,32-23-17-17-5-3-2
1,7.0,BEN-WTM-UJU-PAT-SIB-SHGO-GIR,32-23-17-17-5-3-3
2,4.0,GIR-UJU-WTM-SLGO-SIB-SHGO-SWD-KIS-JSM-SVD,25-15-14-11-9-8-7-6-4-1
3,4.0,GIR-UJU-WTM-SLGO-SIB-SHGO-SWD-KIS-JSM-SVD,25-15-14-11-9-8-7-6-4-1
4,3.5,WTM-GIR-BEN-JSM-PAT-SIB-UJU,30-22-17-17-6-5-2


In [ ]:
df_new = []

for _, row in df.iterrows():
    crude_list = row['Crude diet'].split('-')
    ratio_list = list(map(int, row['Crude ratio'].split('-')))
    
    # Create a dictionary mapping crude names to their ratios
    crude_ratio_dict = dict(zip(crude_list, ratio_list))
    
    # Convert the dictionary to a DataFrame and append to the list
    df_new.append(pd.DataFrame([crude_ratio_dict]))

# Concatenate the DataFrames
result_df = pd.concat(df_new, ignore_index=True)

print(result_df)


      BEN   WTM   UJU   PAT  SIB  SHGO   GIR  SLGO  SWD   KIS  ...  DAR  SUT   
0    32.0  23.0  17.0  17.0    5   3.0   2.0   NaN  NaN   NaN  ...  NaN  NaN  \
1    32.0  23.0  17.0  17.0    5   3.0   3.0   NaN  NaN   NaN  ...  NaN  NaN   
2     NaN  14.0  15.0   NaN    9   8.0  25.0  11.0  7.0   6.0  ...  NaN  NaN   
3     NaN  14.0  15.0   NaN    9   8.0  25.0  11.0  7.0   6.0  ...  NaN  NaN   
4    17.0  30.0   2.0   6.0    5   NaN  22.0   NaN  NaN   NaN  ...  NaN  NaN   
..    ...   ...   ...   ...  ...   ...   ...   ...  ...   ...  ...  ...  ...   
134   NaN   1.0   NaN  19.0    3   1.0  34.0   NaN  NaN   NaN  ...  NaN  NaN   
135   NaN   1.0   NaN  19.0    3   1.0  34.0   NaN  NaN   NaN  ...  NaN  NaN   
136   NaN   1.0   NaN  17.0    2   NaN  44.0   NaN  NaN   NaN  ...  NaN  NaN   
137   NaN   NaN   1.0  15.0    6   NaN  44.0   NaN  NaN   NaN  ...  NaN  NaN   
138   NaN   NaN   NaN   NaN    2   NaN  27.0   NaN  NaN  15.0  ...  NaN  NaN   

      CHA  SSR  MAO  LAB  RBL   MR1  MA

In [ ]:
print(df_new[:5])

[   BEN  WTM  UJU  PAT  SIB  SHGO  GIR
0   32   23   17   17    5     3    2,    BEN  WTM  UJU  PAT  SIB  SHGO  GIR
0   32   23   17   17    5     3    3,    GIR  UJU  WTM  SLGO  SIB  SHGO  SWD  KIS  JSM  SVD
0   25   15   14    11    9     8    7    6    4    1,    GIR  UJU  WTM  SLGO  SIB  SHGO  SWD  KIS  JSM  SVD
0   25   15   14    11    9     8    7    6    4    1,    WTM  GIR  BEN  JSM  PAT  SIB  UJU
0   30   22   17   17    6    5    2]


In [46]:
df_combined = pd.concat(df_new, ignore_index=True)
print(df_combined)

      BEN   WTM   UJU   PAT  SIB  SHGO   GIR  SLGO  SWD   KIS  ...  DAR  SUT   
0    32.0  23.0  17.0  17.0    5   3.0   2.0   NaN  NaN   NaN  ...  NaN  NaN  \
1    32.0  23.0  17.0  17.0    5   3.0   3.0   NaN  NaN   NaN  ...  NaN  NaN   
2     NaN  14.0  15.0   NaN    9   8.0  25.0  11.0  7.0   6.0  ...  NaN  NaN   
3     NaN  14.0  15.0   NaN    9   8.0  25.0  11.0  7.0   6.0  ...  NaN  NaN   
4    17.0  30.0   2.0   6.0    5   NaN  22.0   NaN  NaN   NaN  ...  NaN  NaN   
..    ...   ...   ...   ...  ...   ...   ...   ...  ...   ...  ...  ...  ...   
134   NaN   1.0   NaN  19.0    3   1.0  34.0   NaN  NaN   NaN  ...  NaN  NaN   
135   NaN   1.0   NaN  19.0    3   1.0  34.0   NaN  NaN   NaN  ...  NaN  NaN   
136   NaN   1.0   NaN  17.0    2   NaN  44.0   NaN  NaN   NaN  ...  NaN  NaN   
137   NaN   NaN   1.0  15.0    6   NaN  44.0   NaN  NaN   NaN  ...  NaN  NaN   
138   NaN   NaN   NaN   NaN    2   NaN  27.0   NaN  NaN  15.0  ...  NaN  NaN   

      CHA  SSR  MAO  LAB  RBL   MR1  MA

In [55]:
df_combined = df_combined.fillna(0)
print(df_combined)

      BEN   WTM   UJU   PAT  SIB  SHGO   GIR  SLGO  SWD   KIS  ...  DAR  SUT   
0    32.0  23.0  17.0  17.0    5   3.0   2.0   0.0  0.0   0.0  ...  0.0  0.0  \
1    32.0  23.0  17.0  17.0    5   3.0   3.0   0.0  0.0   0.0  ...  0.0  0.0   
2     0.0  14.0  15.0   0.0    9   8.0  25.0  11.0  7.0   6.0  ...  0.0  0.0   
3     0.0  14.0  15.0   0.0    9   8.0  25.0  11.0  7.0   6.0  ...  0.0  0.0   
4    17.0  30.0   2.0   6.0    5   0.0  22.0   0.0  0.0   0.0  ...  0.0  0.0   
..    ...   ...   ...   ...  ...   ...   ...   ...  ...   ...  ...  ...  ...   
134   0.0   1.0   0.0  19.0    3   1.0  34.0   0.0  0.0   0.0  ...  0.0  0.0   
135   0.0   1.0   0.0  19.0    3   1.0  34.0   0.0  0.0   0.0  ...  0.0  0.0   
136   0.0   1.0   0.0  17.0    2   0.0  44.0   0.0  0.0   0.0  ...  0.0  0.0   
137   0.0   0.0   1.0  15.0    6   0.0  44.0   0.0  0.0   0.0  ...  0.0  0.0   
138   0.0   0.0   0.0   0.0    2   0.0  27.0   0.0  0.0  15.0  ...  0.0  0.0   

      CHA  SSR  MAO  LAB  RBL   MR1  MA

In [56]:
df_merged = pd.concat([df, df_combined], axis=1)

In [57]:
print(df_merged)

           DATE  IOW(In/Out)    pH  Ammonia/ppm  Sulfide/ppm  Chloride/ppm   
0    2022-12-06            0   5.5          2.0        0.034        1.0200  \
1    2022-12-07            0   5.5          9.0        0.088        1.0280   
2    2022-12-22            1  8.58         18.0        0.070        0.7090   
3    2022-12-23            1  8.58         18.0        0.070        0.7090   
4    2022-12-25            1  8.35         56.0        0.120        1.7564   
..          ...          ...   ...          ...          ...           ...   
134  2023-08-27            0  5.87         71.0        2.000        6.2000   
135  2023-08-28            0  6.67         30.0        0.020        6.9127   
136  2023-08-30            0  7.98         52.0        0.060        6.2037   
137  2023-09-01            1  8.51         74.0        0.040        4.4312   
138  2023-09-04            1     9         90.0        0.020        6.9127   

     Iron/ppm  Oil&Grease/ppm  DO/ppm   
0       0.500         

In [58]:
excel_file = "export01.xlsx"
df_merged.to_excel(excel_file, index=False)

In [59]:
# Split data
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, shuffle=True)

NameError: name 'X' is not defined

: 

In [ ]:
clf = DecisionTreeClassifier()

# Step 5: Train the Decision Tree classifier on the training data
clf.fit(X_train, Y_train)

# Step 6: Make predictions on the testing data
Y_pred = clf.predict(X_test)

# Step 7: Evaluate the model's performance
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Accuracy: {accuracy:.2f}")

# You can also print a classification report for more detailed metrics
# classification_rep = classification_report(Y_test, Y_pred, target_names="Out IOW")
# print("Classification Report:\n", classification_rep)

In [ ]:
# Find correlation
# df = df[['H+','Ammonia/ppb', 'Sulfide/ppb', 'Chloride/ppb']]
df = df[['pH','Ammonia/ppm', 'Sulfide/ppm', 'Chloride/ppm','']]
correlation_matrix = df.corr()
correlation_matrix
# Create a correlation heatmap
plt.figure(figsize=(9, 7))  # Set the size of the heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)



# Show the plot
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
#Regression
model = LinearRegression()
model.fit(X_train, Y_train)


In [ ]:
# Make predictions
Y_pred = model.predict(X_test)
# sns.scatterplot(data = df, x = "Y_test", y='Y_pred')
# plt.show()



In [ ]:
# Evaluate the model
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
r1 = r2_score(Y_test, Y_pred)

In [ ]:
print("Evaluation")

print("MSE",mse)
print("RMSE",rmse)
print(f'R-squared : {r1}')

In [ ]:
#Scale data
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#ทำ Ridge Regularization เพื่อ improve model
#ทำ Hyperparameter tuning บน training set โดยใช้ k-fold crossvalidation เพื่อหา best hyperparameter

reg = RidgeCV(alphas=np.logspace(-10, 10, 1000),
              cv=KFold(n_splits=10, shuffle=True),
              scoring='neg_root_mean_squared_error')
reg.fit(X_train_scaled, Y_train)
print("Best alpha : ", reg.alpha_ )

In [ ]:
#นำ best hyperparameter มา train บน training set
ridge = Ridge(alpha=reg.alpha_)
ridge.fit(X_train_scaled, Y_train)
print("Ridge Regression: \n")
print("Mean squared error (MSE) = {}".format(metrics.mean_squared_error(Y_test, ridge.predict(X_test_scaled))))
print(f'Root mean squared error (RMSE):{np.sqrt(metrics.mean_squared_error(Y_test, ridge.predict(X_test_scaled)))}')
y2_pred = ridge.predict(X_test_scaled)
r2 = r2_score(Y_test, y2_pred)
print(f'R-squared : {r2}')

In [ ]:
#ทำ Lasso Regularization เพื่อ improve model
#ทำ Hyperparameter tuning บน training set โดยใช้ k-fold crossvalidation เพื่อหา best hyperparameter
lasso = LassoCV(alphas=np.logspace(-10, 10, 1000),
              cv=KFold(n_splits=10, shuffle=True))
lasso.fit(X_train_scaled, Y_train)
print("Best alpha : ", lasso.alpha_)

In [ ]:
#นำ best hyperparameter มา train บน training set
lasso_a = Lasso(alpha=lasso.alpha_)
lasso_a.fit(X_train_scaled, Y_train)
print("Lasso Regression: \n")
print("Mean squared error (MSE) = {}".format(metrics.mean_squared_error(Y_test, lasso_a.predict(X_test_scaled))))
print(f'Root mean squared error (RMSE): {np.sqrt(metrics.mean_squared_error(Y_test, lasso_a.predict(X_test_scaled)))}')
y_pred = lasso_a.predict(X_test_scaled)
r = r2_score(Y_test, y_pred)
print(f'R-squared : {r}')

In [ ]:
#ทำ Exponential Regression model โดย improve model ด้วย Lasso Regularization
degree = 2

# Standardize the features
scaler = StandardScaler()
X_train_poly = scaler.fit_transform(X_train)
X_test_poly = scaler.transform(X_test)

# Create a Polynomial Features transformer
poly_features = PolynomialFeatures(degree=degree)
X_train_poly = poly_features.fit_transform(X_train_poly)
X_test_poly = poly_features.fit_transform(X_test_poly)


# Create and train the Lasso Regression model
lasso_poly = LassoCV(alphas=np.logspace(-10, 10, 1000),
              cv=KFold(n_splits=10, shuffle=True))
lasso_poly.fit(X_train_poly, Y_train)

lasso_polyb = Lasso(alpha=lasso_poly.alpha_)
lasso_polyb.fit(X_train_poly, Y_train)

print('Lasso Exponential Regression: \n')

print("Best alpha = {}, MSE = {}".format(lasso_poly.alpha_, metrics.mean_squared_error(Y_test, lasso_polyb.predict(X_test_poly))))
print(f'RMSE = {np.sqrt(metrics.mean_squared_error(Y_test, lasso_polyb.predict(X_test_poly)))}')
y4_pred = lasso_polyb.predict(X_test_poly)
r4 = r2_score(Y_test, y4_pred)
print(f'R-squared : {r4}')

In [ ]:
#ทำ Exponential Regression model โดย improve model ด้วย Ridge Regularization
degree = 2
# Standardize the features
scaler_r = StandardScaler()
X_train_poly_r = scaler_r.fit_transform(X_train)
X_test_poly_r = scaler_r.fit_transform(X_test)

# Create a Polynomial Features transformer
poly_features_r = PolynomialFeatures(degree=degree)
X_train_poly_r = poly_features_r.fit_transform(X_train_poly_r)
X_test_poly_r = poly_features_r.fit_transform(X_test_poly_r)

# Create and train the Lasso Regression model
ridge_poly = RidgeCV(alphas=np.logspace(-10, 10, 1000),
              cv=KFold(n_splits=10, shuffle=True))
ridge_poly.fit(X_train_poly_r, Y_train)

lin_poly = Ridge(alpha=ridge_poly.alpha_)
lin_poly.fit(X_train_poly_r, Y_train)

print('Ridge Exponential Regression: \n')

print("Best alpha = {}, MSE = {}".format(ridge_poly.alpha_, metrics.mean_squared_error(Y_test, lin_poly.predict(X_test_poly_r))))
print(f'RMSE = {np.sqrt(metrics.mean_squared_error(Y_test, lin_poly.predict(X_test_poly_r)))}')
y5_pred = lin_poly.predict(X_test_poly_r)
r5 = r2_score(Y_test, y5_pred)

print(f'R-squared : {r5}')